In [15]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
import sys
import re
import shutil
import argparse
import logging
import ast
import itertools
from StringIO import StringIO

import numpy as np
import pandas as pd
from skimage import io, filters, measure, feature, exposure, color, morphology, draw
from skimage.feature import register_translation
from skimage import transform as tf
import fabio as fb
import mahotas as mh

import scipy.ndimage as ndi

from timeit import default_timer as timer

import warnings
warnings.filterwarnings('ignore')

In [16]:
import timeit

_MEASUREMENTS = {
    'Label': 'label',
    'Area': 'area',
    'Perimeter': 'perimeter'
}

_MEASUREMENTS_VALS = _MEASUREMENTS.values()

In [17]:
#https://github.com/zenr/ippy

"""
Implements Kapur-Sahoo-Wong (Maximum Entropy) thresholding method
Usage: $ python max_entropy.py <gray scale image>
"""


def max_entropy(data):
    """
    Implements Kapur-Sahoo-Wong (Maximum Entropy) thresholding method
    Kapur J.N., Sahoo P.K., and Wong A.K.C. (1985) "A New Method for Gray-Level Picture Thresholding Using the Entropy
    of the Histogram", Graphical Models and Image Processing, 29(3): 273-285
    M. Emre Celebi
    06.15.2007
    Ported to ImageJ plugin by G.Landini from E Celebi's fourier_0.8 routines
    2016-04-28: Adapted for Python 2.7 by Robert Metchev from Java source of MaxEntropy() in the Autothresholder plugin
    http://rsb.info.nih.gov/ij/plugins/download/AutoThresholder.java
    :param data: Sequence representing the histogram of the image
    :return threshold: Resulting maximum entropy threshold
    """

    # calculate CDF (cumulative density function)
    cdf = data.astype(np.float).cumsum()

    # find histogram's nonzero area
    valid_idx = np.nonzero(data)[0]
    first_bin = valid_idx[0]
    last_bin = valid_idx[-1]

    # initialize search for maximum
    max_ent, threshold = 0, 0

    for it in range(first_bin, last_bin + 1):
        # Background (dark)
        hist_range = data[:it + 1]
        hist_range = hist_range[hist_range != 0] / cdf[it]  # normalize within selected range & remove all 0 elements
        tot_ent = -np.sum(hist_range * np.log(hist_range))  # background entropy

        # Foreground/Object (bright)
        hist_range = data[it + 1:]
        # normalize within selected range & remove all 0 elements
        hist_range = hist_range[hist_range != 0] / (cdf[last_bin] - cdf[it])
        tot_ent -= np.sum(hist_range * np.log(hist_range))  # accumulate object entropy

        # find max
        if tot_ent > max_ent:
            max_ent, threshold = tot_ent, it

    return threshold

In [18]:
def tryeval(val):
    try:
        val = ast.literal_eval(val)
    except ValueError:
        pass
    return val

In [19]:
def clear_dir(dir_path):
    filelist = [f for f in os.listdir(dir_path) if f.endswith('.tif')]
    for f in filelist:
        os.remove(os.path.join(dir_path, f))

In [20]:
def write_as_raw(data, sample_name, output_dir, prefix=None):
    bits = -1
    if data.dtype == np.int32 or data.dtype == np.float32:
        bits = 32
    elif data.dtype == np.uint8 or data.dtype == np.bool:
        bits = 8

    size = data.shape[::-1]
    output_filename = '{0}_{1}bit_{2}x{3}x{4}.raw'.format(sample_name, bits, *size) if prefix is None \
                        else '{0}_{1}_{2}bit_{3}x{4}x{5}.raw'.format(sample_name, prefix, bits, *size)
    data.tofile(os.path.join(output_dir, output_filename))

In [21]:
def open_gdocs(url):
    import requests
    r = requests.get(url)
    data = r.content
    df = pd.read_csv(StringIO(data))
    return df

In [22]:
def get_samples_configs(df):
    columns = {'Offsets (start, end)': 'offset', 
               'Slice bounding box(x, y, w, h)': 'bbox', 
               'Analysis': 'analysis_type', 
               'Type': 'structure', 
               'Radius': 'radius', 
               'Slice folder': 'slice_folder',
               'Particles density': 'density', 
               'Center': 'center', 
               'Priority': 'priority'}

    cols = df.columns[df.columns.isin(columns.keys())]
    
    out = dict.fromkeys(df['Name'].values, None)
    for name in df['Name'].values:
        out[name] = df[df['Name'] == name][cols].to_dict(orient='records')
        out[name] = {columns[k]: tryeval(v) for k, v in out[name][0].items()}
    
    return out

In [23]:
def object_counter(binary_data):
    labeled_stack, num_labels = ndi.measurements.label(binary_data)
    objects_stats = pd.DataFrame(columns=_MEASUREMENTS_VALS)

    labeled_stack_expanded = labeled_stack[np.newaxis,:,:] \
                                if len(labeled_stack.shape) == 2 else labeled_stack

    for labeled_slice in labeled_stack_expanded:
        for region in measure.regionprops(labeled_slice):
            objects_stats = objects_stats.append({_measure: region[_measure] \
                                            for _measure in _MEASUREMENTS_VALS}, \
                                                ignore_index=True)

    objects_stats = objects_stats.groupby('label', as_index=False).sum()

    return objects_stats, labeled_stack

In [25]:
def filter_particles(data, max_area=2000):
    data = data.copy()
    stats, labels = object_counter(data)
    stats = stats[stats['area'] > max_area]
    for index, row in stats.iterrows():
        labels[labels == row['label']] = 0
        
    labels[np.nonzero(labels)] = 1
    
    return labels

In [26]:
def eliminate_structure_holes(mask, max_area=2000):
    data_mask = ndi.morphology.binary_closing(mask, structure=morphology.disk(2), iterations=3)
    
    data_mask_filled = data_mask.copy()
    data_mask_filled = ndi.morphology.binary_fill_holes(data_mask_filled)

    mask_diff = data_mask_filled - data_mask

    data_labeled = filter_particles(mask_diff, max_area=max_area)
    data_labeled = np.logical_or(data_labeled, data_mask).astype(np.uint8)
    
    return data_labeled

In [12]:
def create_mask_with_circle_ROI(data, radius):
    masked_data = mask_data_with_circle(data, radius)
    
    p2 = np.percentile(masked_data, 0.01)
    p98 = np.percentile(masked_data, 99.99)

    data_8bit = exposure.rescale_intensity(masked_data, \
                                           in_range=(p2, p98), \
                                           out_range=np.uint8).astype(np.uint8)
    data_8bit = filters.median(data_8bit, \
                               selem=morphology.disk(5))

    th = mh.otsu(data_8bit, ignore_zeros=True)

    mask = data_8bit >= th
    mask = mask.astype(np.uint8)
    mask = ndi.morphology.binary_fill_holes(mask)
    
    return data_8bit, mask

In [13]:
def mask_data_with_circle(data, radius, center=None):
    data_shape = data.shape
    
    if center is None:
        center = [v/2. for v in data_shape]
        
    rr,cc = draw.ellipse(center[1], center[0], radius, radius, shape=data_shape)
    circ_mask_roi = np.zeros(data_shape, dtype=np.uint8)
    circ_mask_roi[rr,cc] = 1
    masked_data = data * circ_mask_roi
    clipped_data = masked_data[(center[1]-radius):(center[1]+radius), \
                               (center[0]-radius):(center[0]+radius)]
    
    return masked_data, clipped_data

In [14]:
def segment_sample_with_circle_ROI(samples_info,
                                   input_dir_tmpl='./data/intelbiocomp_bioglass_raw_data/recon/{sample_name}/{slice_folder}',
                                   output_dir_tmpl='./data/intelbiocomp_bioglass_results', 
                                   median_filter_rad=2):
    
    for sample_name, configs in samples_info.iteritems():
        start = timer()
        
        print 'Sample #' + sample_name
        
        radius = configs['radius']
        atype = configs['analysis_type'].strip()
        slice_folder = configs['slice_folder']
        srgn = configs['offset']
        dsty = configs['density']
        center = configs['center']
        
        components = atype.split('/')
        
        def _create_dir(path):
            if not os.path.exists(path):
                os.makedirs(path)
            else:
                shutil.rmtree(path)
                os.makedirs(path)

            return path
        
        crop_output_dir = _create_dir(os.path.join(output_dir_tmpl, sample_name, 'cropped'))
        clipped_output_dir = _create_dir(os.path.join(output_dir_tmpl, sample_name, 'clipped_roi'))
        
        input_dir = input_dir_tmpl.format(sample_name=sample_name, slice_folder=slice_folder)
        
        print input_dir
        print crop_output_dir
        print clipped_output_dir
        
        filenames = os.listdir(input_dir)
        filenames = [fn for fn in filenames if fn.endswith('.tif')]
        files = [os.path.join(input_dir, f) for f in filenames]
        
        for fpath in itertools.islice(sorted(files), srgn[0], len(files) + srgn[1]):
            data = fb.open(fpath).data
            filename = os.path.basename(fpath)
            
            cropped_data, clipped_data = mask_data_with_circle(data, radius, center=center)
            
            io.imsave(os.path.join(crop_output_dir, filename), cropped_data)
            io.imsave(os.path.join(clipped_output_dir, filename), clipped_data)

        for comp in components:
            output_dir = _create_dir(os.path.join(output_dir_tmpl, sample_name, comp.lower() + '_masks'))

            filenames = os.listdir(crop_output_dir)
            filenames = [fn for fn in filenames if fn.endswith('.tif')]
            files = [os.path.join(crop_output_dir, f) for f in filenames]

            for i, fpath in enumerate(sorted(files)):
                cropped_data = fb.open(fpath).data
                filename = os.path.basename(fpath)

                if i % 250 == 0 or i == (len(files) - 1):
                    print '%d/%d' % (i, len(files) - 1)
                    
                p2 = np.percentile(cropped_data, 0.01)
                p98 = np.percentile(cropped_data, 99.99)
        
                cropped_data = exposure.rescale_intensity(cropped_data, in_range=(p2, p98))
                data_8bit = exposure.rescale_intensity(cropped_data, in_range='image', out_range=np.uint8).astype(np.uint8)
                data_8bit = filters.median(data_8bit, selem=morphology.disk(median_filter_rad))
                
                th = max_entropy(np.histogram(data_8bit, bins=256, range=(0, 256))[0]) if dsty == 'sparse' else \
                        mh.otsu(data_8bit, ignore_zeros=True)
                    
                mask = data_8bit >= th 
                mask = mask.astype(np.uint8)
                
                io.imsave(os.path.join(output_dir, filename), mask.astype(np.uint8))
            
        end = timer()
        print end - start

In [263]:
def calculate_features(samples_info, \
                       input_dir_tmpl='./data/intelbiocomp_bioglass_results', \
                       roi_box=((-175,175),(-175,175),(-175,175))):
    
    roi_side = np.sum(np.abs(roi_box[0]))
    
    def _create_dir(path):
        if not os.path.exists(path):
            os.makedirs(path)
        else:
            shutil.rmtree(path)
            os.makedirs(path)
        
        return path
    
    for sample_name, configs in samples_info.iteritems():
        start = timer()
        
        center = configs['center']
        
        print 'Sample #' + sample_name
        
        atype = configs['analysis_type'].strip()
        components = atype.lower().split('/')
        
        if 'polymer' not in atype.lower() and \
           'particle' not in atype.lower():
                raise ValueError('The sample component(s) are not specified.')
        
        for comp in components: 
            input_dir = os.path.join(input_dir_tmpl, sample_name, comp.lower() + '_masks')
            
            filenames = os.listdir(input_dir)
            files = [os.path.join(input_dir, f) for f in filenames]
            
            try:
                slice_shape = io.imread(files[0]).shape
            except:
                slice_shape = fb.open(files[0]).data.shape
            
            print 'Porosity calculation...'
            output_dir = _create_dir(os.path.join(input_dir_tmpl, sample_name, 'porosity_stats_{}'.format(roi_side)))
                
            if not len(files):
                raise ValueError('No mask files.')

            slice_area = slice_shape[0] * slice_shape[1]
                
            try:
                porosity_comps = [np.count_nonzero(io.imread(fpath)) / float(slice_area) \
                                    for fpath in files]
            except:
                porosity_comps = [np.count_nonzero(fb.open(fpath).data) / float(slice_area) \
                                    for fpath in files]

            pd.DataFrame({'Porosity': [sum(porosity_comps)]}, \
                            index=[sample_name]).to_csv(os.path.join(output_dir, 'porosity.csv'))
                
            print 'Particles counting...'
            output_dir = _create_dir(os.path.join(input_dir_tmpl, sample_name, 'particles_stats_{}'.format(roi_side)))
            
            depth, height, width  = len(files), slice_shape[0], slice_shape[1]
            cd, ch, cw = depth/2, height/2, width/2
                
            if roi_box is not None:
                data_mask_vol = np.zeros(tuple([np.sum(np.abs(v)) for v in roi_box]), dtype=np.uint8)
            else:
                data_mask_vol = np.zeros((depth, height, width), dtype=np.uint8)
            
            path_data = itertools.islice(sorted(files), cd+roi_box[0][0], cd+roi_box[0][1]) if roi_box is not None \
                        else sorted(files)
            
            for i, fpath in enumerate(path_data):
                try:
                    data_slice = io.imread(fpath)
                except:
                    data_slice = fb.open(fpath).data
                    
                data_slice = data_slice[slice(center[1]+roi_box[1][0], center[1]+roi_box[1][1], 1), \
                                        slice(center[0]+roi_box[2][0], center[0]+roi_box[2][1], 1)]
                    
                data_mask_vol[i] = data_slice
                
            print data_mask_vol.shape, data_mask_vol.dtype
        
            particle_stats, labeled_data = object_counter(data_mask_vol)
            particle_stats.to_csv(os.path.join(output_dir, 'particles.csv'))
            
            write_as_raw(labeled_data.astype(np.uint8), sample_name, output_dir)
        
        end = timer()
        print end - start

In [260]:
def segment_sample(samples_configs, \
                   path_template='./data/intelbiocomp_bioglass_raw_data/recon/biomaterials/materials/{0}/tomo1/{1}', \
                   in_folder='slices_phase_particles', \
                   max_area=2500):
    for sample_name, configs in samples_configs.iteritems():
        print 'Sample #' + sample_name
        
        bbox = configs['bbox']
        atype = configs['analysis_type'].strip()
        struct_type = configs['structure']
        
        components = atype.split('/')
        
        data_output_dir = path_template.format(sample_name, 'cropped_data')
        if not os.path.exists(data_output_dir):
            os.makedirs(data_output_dir)
        else:
            shutil.rmtree(data_output_dir)
            os.makedirs(data_output_dir)
            
        input_dir = path_template.format(sample_name, in_folder)
        
        for comp in components:
            output_dir = path_template.format(sample_name, comp.lower() + '_masks')

            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            else:
                shutil.rmtree(output_dir)
                os.makedirs(output_dir)

            filenames = os.listdir(input_dir)
            files = [os.path.join(input_dir, f) for f in filenames]

            working_files = files[configs['offset'][0]: configs['offset'][1]]

            for i, fpath in enumerate(working_files):
                data = io.imread(fpath)

                filename = os.path.basename(fpath)

                if i % 200 == 0 or i == (len(working_files) - 1):
                    print '%d/%d' % (i, len(working_files) - 1)

                data = data[bbox[1]:(bbox[1] + bbox[3] + 1), bbox[0]:(bbox[1] + bbox[2] + 1)]
                p2 = np.percentile(data, 0.01)
                p98 = np.percentile(data, 99.99)
                data8bit = exposure.rescale_intensity(data, in_range=(p2, p98), out_range=np.uint8).astype(np.uint8)
                hist = np.histogram(data8bit, bins=256, range=(0, 256))[0]
            
                if 'particle' in comp.lower():
                    th = max_entropy(hist)
                elif 'polymer' in comp.lower():
                    th = filters.threshold_otsu(data8bit, nbins=256)
                else:
                    raise ValueError('Unknown sample type.')

                mask = data8bit <= th
                mask = mask.astype(np.uint8)
                
                if 'polymer' in comp.lower() and struct_type == 'Thick':
                    mask = eliminate_structure_holes(mask, max_area=max_area)

                io.imsave(os.path.join(output_dir, filename), mask)
                
                cropped_data_path = os.path.join(data_output_dir, filename)
                
                if not os.path.isfile(cropped_data_path):
                    io.imsave(cropped_data_path, data8bit)

In [138]:
def multi_otsu(data):
    hist = np.histogram(data.ravel(), 256)[0]
    size = float(data.size)
    
    th1, th2 = 0, 0
    mt, max_var = 0, 0
    
    for k in xrange(256):
        mt += k * hist[k] / size
        
    w0k, m0k = 0, 0
    
    for t1 in xrange(256):
        w0k += hist[t1] / size
        m0k += t1 * hist[t1] / size
        m0 = m0k / w0k
        
        w1k, m1k = 0, 0
        
        for t2 in xrange(1, 256):
            w1k += hist[t2] / size
            m1k += t2 * hist[t2] / size
            m1 = m1k / w1k
            
            w2k = 1. - (w0k + w1k)
            m2k = mt - (m0k + m1k)
            
            if w2k <= 0:
                break
                
            m2 = m2k / w2k
            
            curr_var = w0k * (m0 - mt) * (m0 - mt) + w1k * (m1 - mt) * (m1 - mt) + w2k * (m2 - mt) * (m2 - mt)
            
            if max_var < curr_var:
                max_var = curr_var
                th1, th2 = t1, t2
    
    print max_var
    return th1, th2